In [26]:
! pip install selenium

  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)


In [3]:
!pip install bs4

In [1]:
import requests
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import os
import re
import pandas as pd
from urllib.parse import urlparse

In [2]:
def address_to_latitude(address):
    global lat
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query='+address
    result = requests.get(urlparse(url).geturl(),
                      headers = {'Authorization': 'KakaoAK {}'.format('012f3277daabe10113b230330bd12a77')})
    json_obj = result.json()
    for document in json_obj['documents']:
        lat = document['y']
    return lat

def address_to_longtitude(address):
    global long
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query='+address
    result = requests.get(urlparse(url).geturl(),
                      headers = {'Authorization': 'KakaoAK {}'.format('012f3277daabe10113b230330bd12a77')})
    json_obj = result.json()
    for document in json_obj['documents']:
         long = document['x']
    return long

In [6]:
search_key='강남역 카페'
options = webdriver.ChromeOptions() # 크롬 브라우저 옵션
options.add_argument('headless') # 브라우저 안 띄우기
options.add_argument('lang=ko_KR') #  KR 언어
driver = webdriver.Chrome('C:/Users/chaeh/chromedriver')
driver.get('https://map.kakao.com/') 
driver.find_element_by_xpath('/html/body/div[10]/div/div/div/strong').click()
search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]') # 검색 창
search_area.send_keys(search_key)# 검색어 입력
driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
driver.implicitly_wait(10)
driver.find_element_by_xpath('//*[@id="info.search.place.sort"]/li[2]/a').click()
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

title=soup.find_all('a', class_='link_name')
li1=[]
for i in range(10):
    li1.append(title[i]['title'])
print(li1)

cat=soup.find_all(class_='subcategory clickable')
li2=[]
pattern='<[^>]*>'
for i in range(10):
    tp=str(cat[i])
    tp=re.sub(pattern=pattern, repl='', string=tp)
    li2.append(tp)
print(li2)

addr=soup.find_all('div',class_='addr')
li3=[]
for i in range(10):
    tp=str(addr[i])
    tp=re.sub(pattern=pattern, repl='', string=tp)
    tp=re.search('\n(.*)\n', tp).group(1)
    li3.append(tp)
print(li3)
#driver.quit() # driver 종료, 브라우저 닫기

li4=[]
li5=[]
for i in range(10):
    li4.append(address_to_latitude(li3[i]))
    li5.append(address_to_longtitude(li3[i]))

    
list=[]
for i in range(10):
    li=[]
    li.append(li1[i])
    li.append(li2[i])
    li.append(li3[i])
    li.append(li4[i])
    li.append(li5[i])
    list.append(li)
df=pd.DataFrame(list,columns=['name','type','addr','lat','lng'])
df

['블루보틀 역삼', '랑데자뷰 강남점', '알베르', '타르타르 강남역점', '정월', '썸띵어바웃커피', '키이스케이프 강남점', '빌리엔젤 강남역점', '겟썸커피', '평화다방 강남역점']
['카페', '카페', '카페', '디저트카페', '카페', '디저트카페', '테마카페', '디저트카페', '커피전문점', '카페']
['서울 강남구 테헤란로 129 강남N타워 1층', '서울 강남구 봉은사로6길 39 1층', '서울 강남구 강남대로102길 34', '서울 서초구 강남대로61길 13 1층', '서울 강남구 강남대로102길 46 1층', '서울 강남구 강남대로102길 30 1~3층', '서울 강남구 강남대로96길 17', '서울 강남구 테헤란로1길 48 강남ELS빌딩 1층', '서울 강남구 봉은사로6길 30 지하1층', '서울 강남구 강남대로94길 10 K스퀘어 2층']


,name,type,addr,lat,lng
0,블루보틀 역삼,카페,서울 강남구 테헤란로 129 강남N타워 1층,37.4999326440603,127.032367803374
1,랑데자뷰 강남점,카페,서울 강남구 봉은사로6길 39 1층,37.5026858061231,127.027942392592
2,알베르,카페,서울 강남구 강남대로102길 34,37.5030565318496,127.028103011462
3,타르타르 강남역점,디저트카페,서울 서초구 강남대로61길 13 1층,37.4988914895844,127.02555561464
4,정월,카페,서울 강남구 강남대로102길 46 1층,37.5036597194557,127.028626641848
5,썸띵어바웃커피,디저트카페,서울 강남구 강남대로102길 30 1~3층,37.5036597194557,127.028626641848
6,키이스케이프 강남점,테마카페,서울 강남구 강남대로96길 17,37.5001355825173,127.028389730314
7,빌리엔젤 강남역점,디저트카페,서울 강남구 테헤란로1길 48 강남ELS빌딩 1층,37.5014643340657,127.027076332559
8,겟썸커피,커피전문점,서울 강남구 봉은사로6길 30 지하1층,37.5030771832102,127.027255717013
9,평화다방 강남역점,카페,서울 강남구 강남대로94길 10 K스퀘어 2층,37.4991963225538,127.028227205943


맥도날드나 스타벅스등 중복값 제거는 ? -> 어차피 인기순이라서 그렇게 겹치는것도 없고, 서울시 한정이라 겹치는 것 많이 없음

type는 왜 통일 안했을까? -> 어차피 검색할때 00역 카페, 00역 햄버거 이렇게 하면 비슷한 장소 다 도출해줌! (카카오맵이편하다...)

입력값: 자세히할수록 좋은듯! ex) 강남역 10번출구 근처 카페 (좀 지리상 멀어지긴 하더라도 거기가 인기있는 곳이니까 더 의미있을것이라고 생각)

200510 마지막수정